In [56]:
!pip install PyDrive
!pip install XlsxWriter

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1QtoxpJmd1lys7c7LaYXiOjbzMdMOpeVX'})
download.GetContentFile('1998.tar')

In [0]:
download2 = drive.CreateFile({'id': '1xaJL1eoccrCyS45xgF23dVY_KCER-oAD'})
download2.GetContentFile('trec07p.tar')

In [0]:
!tar xf 1998.tar
!tar xf trec07p.tar

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [0]:
notHeaders = ['A',
 'A.normal2{color',
 'A.normal{color',
 'A.over{color',
 'ATT',
 'Action',
 'Added',
 'Address',
 'An',
 'Asunto',
 'Author',
 'Auto-Submitted',
 'BATCH',
 'Betreff',
 'Beurteilung',
 'Bookmark',
 'Brma',
 'BrmaSmtpAuthUser',
 'C',
 'Call',
 'Cancel-Lock',
 'Carlson',
 'Caveats',
 'Changeset',
 'Cia1iis',
 'Cited',
 'Classification',
 'Comment',
 'Company',
 'Complaints',
 'Congratulations',
 'Contact',
 'Copyright',
 'Corp.<http',
 'Corporation<http',
 'Credibility',
 'Date-warning',
 'Datum',
 'De',
 'Details',
 'De\xa0',
 'Diagnostic-Code',
 'Dinah',
 'Envoyé\xa0',
 'FATAL',
 'FDA',
 'FONT-SIZE',
 'Featuring',
 'Fixes',
 'Gesendet',
 'Hash',
 'Hinweis',
 'ISIN',
 'Importance',
 'Inc.<http',
 'Index',
 'Jabber-ID',
 'Jim',
 'John',
 'Kopia',
 'LINE-HEIGHT',
 'List-Help',
 'Log',
 'London',
 'Lookup',
 'MOTD',
 'Mail-Followup-To',
 'Managed-by',
 'Market',
 'Metze',
 'Modified',
 'NOTE',
 'Name',
 'Napster<http',
 'Newshawk',
 'Notice',
 'Number',
 'OTC',
 'Objet\xa0',
 'Old-Return-Path',
 'OpenPGP',
 'Organisation',
 'Organization',
 'PADDING-BOTTOM',
 'PHONE',
 'Package',
 'Page',
 'Para',
 'Phone',
 'Posted',
 'Precedence',
 'President',
 'Priority',
 'Products',
 'Pubdate',
 'Publicitate',
 'REF',
 'RT-Ticket',
 'Rangel',
 'Received-SPF',
 'References',
 'Reminder',
 'Removed',
 'Reporting-MTA',
 'S.umbol',
 'Sent',
 'Severity',
 'Site',
 'Skickat',
 'Source',
 'Spoken',
 'Sym8oL',
 'Symbol',
 'Sys.putenv("http_proxy"="http',
 'TELEPHONE',
 'Talk',
 'Tel',
 'Teste',
 'Ticker',
 'Till',
 'Timing',
 'Tname',
 'Try',
 'Type',
 'UEI',
 'URL',
 'Visit',
 'Von',
 'WASHINGTON',
 'WKN',
 'Webpage',
 'Website',
'A.RVTS2',
 'ASTIG',
 'AXIS',
 'Ambieen',
 'Anova',
 'Below',
 'CARGO',
 'CNN',
 'CNNMoney',
 'City',
 'Clinton',
 'Collins',
 'Given',
 'HEADLINES',
 'IRAN',
 'Iterations',
 'Juego!<o',
 'Kernel',
 'L<Parrot',
 'Parrot',
 'Price']

In [0]:
def parseTextFromHTML(body):
    body = body.lower()
    htmlstartindex = body.find('<html>')
    htmlendindex = body.find('</html>')
    
    #if(htmlstartindex == -1 or htmlendindex == -1):
        #return body
    
    htmlcontent = body#[htmlstartindex:htmlendindex+7]
    soup = BeautifulSoup(htmlcontent, 'html.parser')
    [s.extract() for s in soup(['style', 'script'])]
    return soup.getText().strip()
    #return body[:htmlstartindex] + soup.getText().strip() + body[htmlendindex+7:]

In [65]:
"""# parse the email file(spam or ham) to create a pandas dataframe
def getDFFromEmail(path, spam):
    l = []
    precurrkv = []
    sufcurrkv = []
    count = 0
    body = False
    try:
        stream = open(path, errors='strict', encoding='UTF-8')
        stream.readlines()
        stream.seek(0)
        print("Using UTF-8 encoding")
    except (Exception):
        print("Using ANSI encoding")
        stream = open(path, errors='strict', encoding='ANSI')
        stream.readlines()
        stream.seek(0)
        
    for line in stream:
        if(line.startswith('<DOCTYPE')):
            continue
            
        colonIndex = line.find(":")
        
        if(count > 15 and not body):
            body = True
            l.append(precurrkv)
            
        if(line[0].isupper() and line.find(' ',0,colonIndex)<0 and colonIndex >= 0 and colonIndex < 30 and not body 
           and line[:colonIndex] not in notHeaders):
            count = 0
            if(len(sufcurrkv) > 0 and len(precurrkv) > 0):
                precurrkv[1] = "".join((precurrkv[1], "".join(sufcurrkv)))
                l.append(precurrkv)
            elif(len(precurrkv) > 0):
                l.append(precurrkv)
            precurrkv = line.split(sep=":", maxsplit=1)
            sufcurrkv.clear()

        elif(str(line[0:2]).isspace() and not body):
            count += 1
            sufcurrkv.append(line)
        elif(str(line).startswith('>') and not body):
            body = True
            sufcurrkv = [''.join((':'.join(precurrkv), "".join(sufcurrkv)))]
        else:
            #count += 1
            sufcurrkv.append(line)

    l.append(["Body", parseTextFromHTML("".join(sufcurrkv))])
    l.append(["Spam", 'Spam' if spam else 'Ham'])
    l.append(["Tname", path])

    d = pd.DataFrame(np.array(l)).drop_duplicates(subset=0)
    return d    """
""""""

''

In [0]:
# parse the email file(spam or ham) to create a pandas dataframe
from collections import deque
from codecs import open
def getDFFromEmail(path, spam):
    l = dict()
    precurrkv = []
    sufcurrkv = deque()
    count = 0
    body = False
        
    try:
        stream = open(path, errors='strict')
        stream.readlines()
        stream.seek(0)
        print("Using UTF-8 encoding")
    except (Exception):
        try:
            print("Using ISO-8859-1 encoding")
            stream = open(path, errors='strict', encoding='iso-8859-1')
            stream.readlines()
            stream.seek(0)
        except (Exception):
            return ""
        
    for line in stream:
        if(line.startswith('<DOCTYPE')):
            continue
            
        colonIndex = line.find(":")
        
        if(count > 15 and not body):
            body = True

            
        if(line[0].isupper() and line.find(' ',0,colonIndex)<0 and colonIndex >= 0 and colonIndex < 30 and not body 
           and line[:colonIndex] not in notHeaders):
            count = 0
            if(sufcurrkv and len(precurrkv) > 0):
                fieldname = sufcurrkv.popleft()
                l[fieldname] = [''.join(sufcurrkv)]
            
            precurrkv = line.split(sep=":", maxsplit=1)
            sufcurrkv.clear()
            sufcurrkv.extend(precurrkv)

        elif(str(line[0:2]).isspace() and not body):
            count += 1
            sufcurrkv.append(line)
        elif(str(line).startswith('>') and not body):
            body = True
            sufcurrkv.append(line)
        else:
            #count += 1
            sufcurrkv.append(line)

    fieldname = sufcurrkv.popleft()
    l[fieldname] = sufcurrkv.popleft()    
    l["Body"] = [parseTextFromHTML("".join(sufcurrkv))]
    l["Spam"] = ['Spam'] if spam else ['Ham']
    l["Tname"] = [path]

    d = pd.DataFrame(l)
    return d.T.reset_index()    

In [0]:
# iterate over all files in the directory to create the dataframes
def getAllDFFromDirectory(directorypath):
    for filename in os.listdir(directorypath):
        print("Getting DF for "+"".join((directorypath, filename)))
        if(filename.startswith("spm")):
            yield getDFFromEmail("".join((directorypath, filename)), True)
        elif(filename.count("msg") > 0):
            yield getDFFromEmail("".join((directorypath, filename)), False)
        else:
            yield getDFFromEmail("".join((directorypath, filename)), True)

In [0]:
from functools import reduce

# create dataframes from all directories from that contain spam or ham email text files

In [69]:
directorypath = "1998/1998/03/"
dfs = [i for i in getAllDFFromDirectory(directorypath) if(isinstance(i,pd.DataFrame))]

Getting DF for 1998/1998/03/890929492.24871.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929572.24886.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929472.24865.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929566.24884.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/891219215.5425.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929468.24864.txt
Using ISO-8859-1 encoding
Getting DF for 1998/1998/03/890929499.24873.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929475.24866.txt
Using ISO-8859-1 encoding
Getting DF for 1998/1998/03/891285044.11747.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/890929496.24872.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/891002827.28090.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/891020038.3226.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/891219148.5406.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03/891184729.10913.txt
Using UTF-8 encoding
Getting DF for 1998/1998/03

In [0]:
# iterate over all files in the directory to create the dataframes
def getAllDFFromDirectory2(directorypath, target):
    for i,filename in enumerate(os.listdir(directorypath)[:15000]):
        print("Getting DF for "+"".join((directorypath, filename)))
        
        if(target[i] == 'spam'):
            yield getDFFromEmail("".join((directorypath, filename)), True)
        else:
            yield getDFFromEmail("".join((directorypath, filename)), False)

In [71]:
def getTrec07Target():
    for line in open("trec07p/trec07p/full/index"):
        yield line.split(" ")[0]

target = [i for i in getTrec07Target()]
print(len(target))

directorypath = "trec07p/trec07p/data/"
dfs2 = [i for i in getAllDFFromDirectory2(directorypath, target) if(isinstance(i,pd.DataFrame))]

75419
Getting DF for trec07p/trec07p/data/inmail.43936
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.45889
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.57941
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.65165
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.22060
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.331
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.51819
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.31323
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.60852
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.58300
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.73043
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.1414
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.824
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.59314
Using UTF-8 encoding
Gett

Getting DF for trec07p/trec07p/data/inmail.61755
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.23054
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.68229
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.60555
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.27789
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.40955
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.13880
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.5514
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.3444
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.13671
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.35797
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.21286
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.31552
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.42753
Using UTF-8 encoding
Getting

Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.40168
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.41900
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.18253
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.11477
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.22723
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.64890
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.65923
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.37828
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.18205
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.12957
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.13800
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.28512
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.47446
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.19381
Using UTF-


Getting DF for trec07p/trec07p/data/inmail.9265
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.73080
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.54429
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.62052
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.65828
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.3576
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.42671
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.69885
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.51641
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.57179
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.31636
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.59127
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.43246
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.58019
Using UTF-8 encoding
Getting DF 

Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.10793
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.7972
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.50244
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.268
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.61162
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.13673
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.66288
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.64364
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.75057
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.48161
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.52837
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.46887
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.54869
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.50881
Using UT

...

Getting DF for trec07p/trec07p/data/inmail.23354
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.27902
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.32605
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.52896
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.10156
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.37496
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.58387
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.37973
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.65205
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.23235
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.2148
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.19316
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.48563
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.41494
Using UTF-8 encoding
Getting DF for t


Getting DF for trec07p/trec07p/data/inmail.35077
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.38640
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.9556
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.1294
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.30081
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.44132
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.68675
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.37825
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.30759
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.25326
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.21563
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.33835
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.65614
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.29606
Using UTF-8 encoding
Getting DF for t


Getting DF for trec07p/trec07p/data/inmail.47947
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.68018
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.27087
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.71717
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.29691
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.54179
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.41137
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.53862
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.44458
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.48371
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.60414
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.1259
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.75065
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.35317
Using UTF-8 encoding
Getting DF


Getting DF for trec07p/trec07p/data/inmail.53248
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.74888
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.72260
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.57128
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.66168
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.7235
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.41492
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.11619
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.305
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.13140
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.49530
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.17576
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.218
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.6827
Using ISO-8859-1 encoding


Getting DF for trec07p/trec07p/data/inmail.7639
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.67113
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.75415
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.38807
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.48111
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.70169
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.20170
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.41784
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.9322
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.71744
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.27990
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.27646
Using UTF-8 encoding
Getting DF for trec07p/trec07p/data/inmail.68790
Using ISO-8859-1 encoding
Getting DF for trec07p/trec07p/data/inmail.22557
Using UTF-8 encoding
Getting

In [0]:
# Test for number of columns in dataframes
#dfs2.map(lambda df: df.shape[0])
#[i for i in zip(map(lambda df: df.shape[0],dfs2), range(len(dfs2))) if(i[0]>40)]


In [0]:
df_final = reduce(lambda left,right: pd.merge(left,right,how='outer',on='index'), dfs+dfs2)

In [0]:
df_final = df_final.set_index('index').T

In [74]:
df_final['index'] = [i for i in range(df_final.shape[0])]
df_final.set_index('index', inplace=True)
df_final.head()

index,Body,Cc,Date,Delivered-To,From,Message-ID,Received,Return-Path,Spam,Tname,...,ÁªÏµ,Ö÷½²¿Î³Ì,X-M,L.P.<http,X-Eon-Dm,X-Eon-Sig,X-SpamDetect,Conf,X-campaignid,X-SGUL-MailScanner-MH1
index,,,,,,,,,,,,,,,,,,,,,
0,"baileyli@hubble.sheridanc.on.ca, baileym@conce...","baileyl@cream.cambridge.scr.slb.com, baileyl@...","Thu, 12 Mar 1998 17:23:10 EST\n",bait@mikhail.qcc.sk.ca\n,MBran <MBran@aol.com>\n,<367a6bc2.35086056@aol.com>\n,from MBran@aol.com\n\tby imo14.mx.aol.com (IM...,<MBran@aol.com>\n,Spam,1998/1998/03/890929492.24871.txt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,attention! rca introduces internet from your t...,NaN,NaN,tech@qcc.sk.ca\n,<vworld@gigabizops.com>\n,NaN,from ppp-207.115.33.46.prodigy.net (HELO serv...,<vworld@gigabizops.com>\n,Spam,1998/1998/03/890929572.24886.txt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,>>>this is the most exciting breakthrough ever...,NaN,"Wed, 04 Feb 98 00:14:47 EST\n",bait@mikhail.qcc.sk.ca\n,iwbp@mailcity.com\n,<>\n,from mail.hic.net (1Cust113.tnt8.lax3.da.uu.n...,<iwbp@mailcity.com>\n,Spam,1998/1998/03/890929472.24865.txt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,earn $500.00 by noon tomorrow!\n\njust refer p...,NaN,"Wed, 25 Mar 98 08:24:47 EST\n",bguenter@hal.qcc.sk.ca\n,72334890@bigfoot.com\n,<199803140024.GAA02943@1arbiscad.com>\n,from mailhost.1arbiscad.com (alt2.1arbiscad.c...,<72334890@bigfoot.com>\n,Spam,1998/1998/03/890929566.24884.txt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,**** water is the most essential element for s...,NaN,"Sun, 29 Mar 98 02:46:14 EST\n",bguenter@hal.qcc.sk.ca\n,41480152@29086.com\n,NaN,from host (PhoenixAZDP84-95.SplitRock.net [20...,<41480152@29086.com>\n,Spam,1998/1998/03/891219215.5425.txt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def combineDuplicateColumns(df_final, colNames):
    originalCol = ''
    for i,duplicateCol in enumerate(colNames):
        if(i == 0):
            originalCol = duplicateCol
            continue
        
        df_final.loc[df_final[originalCol].isna(), originalCol] = df_final.loc[df_final[originalCol].isna()][duplicateCol]
    
    
    df_final.drop(labels=colNames[1:], axis=1, inplace=True)

In [0]:
# Combine duplicate columns
combineDuplicateColumns(df_final, ['Message-Id', 'Message-ID', 'Message-id'])
combineDuplicateColumns(df_final, ['Reply-To', 'Reply-to'])
combineDuplicateColumns(df_final, ['Mime-Version', 'MIME-version', 'MIME-Version'])
combineDuplicateColumns(df_final, ['Content-Type', 'Content-type'])
combineDuplicateColumns(df_final, ['Content-Transfer-Encoding', 'Content-transfer-encoding'])
combineDuplicateColumns(df_final, ['Error-To', 'Errors-To', 'Errors-to'])
combineDuplicateColumns(df_final, ['Content-Length','Content-length'])

In [77]:
# Check for more duplicate columns
np.sort(df_final.columns.values)

array(['A.cantseetext', 'AAA', 'AAA1', ..., 'ÏêÇéÇëµÇÂ½http', 'Ò»',
       'Ö÷½²¿Î³Ì'], dtype=object)

In [78]:
#From, To, Body, Subject, Message-Id, X-UIDL, Sender, Spam
c1 = df_final.columns
c2 = pd.Index(['Spam', 'Body', 'Subject', 'From', 'To', 'Message-Id', 'X-UIDL', 'Sender'])
c1 = c1.difference(c2)

df_new = df_final.drop(labels=c1.format(), axis=1, inplace=False)
df_new.columns

Index(['Body', 'From', 'Spam', 'To', 'Subject', 'X-UIDL', 'Message-Id',
       'Sender'],
      dtype='object', name='index')

In [79]:
excelwriter = pd.ExcelWriter('structured.xlsx', engine='xlsxwriter')

df_new.to_excel(excelwriter, index=False)

excelwriter.save()

/usr/local/lib/python3.6/dist-packages/xlsxwriter/worksheet.py:832: UserWarning: Ignoring URL 'http://agreeput.hk




viagra

30%20pills%20x%2050mg$60.23$2.01%20per%20itemyour%20save:%20$3060%20pills%20x%2050mg$111.65$1.86%20per%20itemyour%20save:%20$7010%20pills%20x%20100mg$34.49$3.45%20per%20item%2030%20pills%20x%20100mg$88.5$2.95%20per%20itemyour%20save:%20$1560%20pills%20x%20100mg$141$2.35%20per%20itemyour%20save:%20$6690%20pills%20x%20100mg$176.4$1.96%20per%20itemyour%20save:%20$13410%20pills%20x%2050mg$30.22$3.03%20per%20item%20

viagra%20%20soft%20tabs


%2010%20pills%20x%2050mg%20%20$33.25%20%20$3.33%20per%20item%20%20%20%20%2030%20pills%20x%2050mg%20%20$66.28%20%20$2.21%20per%20item%20%20your%20save:%20$34%20%2060%20pills%20x%2050mg%20%20$123.2%20%20$2.05%20per%20item%20%20your%20save:%20$77%20%2010%20pills%20x%20100mg%20%20$37.94%20%20$3.8%20per%20item%20%20%20%20%2030%20pills%20x%20100mg%20%20$97.35%20%20$3.25%20per%20item%20%20your%20save:%20$16%20%2060%20pills%20x%20100mg%

/usr/local/lib/python3.6/dist-packages/xlsxwriter/worksheet.py:832: UserWarning: Ignoring URL 'http://temperaturegave.hk


viagra


30%20pills%20x%20100mg$88.5$2.95%20per%20itemyour%20save:%20$1510%20pills%20x%20100mg$34.49$3.4=
5%20per%20item%2060%20pills%20x%20100mg$141$2.35%20per%20itemyour%20save:%20$6660%20pills%20x%2050mg$=
111.65$1.86%20per%20itemyour%20save:%20$7030%20pills%20x%2050mg$60.23$2.01%20per%20itemyour%20s=
ave:%20$3090%20pills%20x%20100mg$176.4$1.96%20per%20itemyour%20save:%20$13410%20pills%20x%2050mg$=
30.22$3.03%20per%20item%20


viagra%20soft%20tabs

%2060%20pills%20x%2050mg%20%20$123.2%20%20$2.05%20per%20item%20%20your%20save:%20$77%20%2030%20pills%20x%2050mg%20%20=
$66.28%20%20$2.21%20per%20item%20%20your%20save:%20$34%20%2010%20pills%20x%20100mg%20%20$37.94%20%20$3.8%20per%20=
item%20%20%20%20%2030%20pills%20x%20100mg%20%20$97.35%20%20$3.25%20per%20item%20%20your%20save:%20$16%20%2060%20pills=
%20x%20100mg%20%20$155.1%20%20$2.59%20per%20item%20%20your%20save:%2

/usr/local/lib/python3.6/dist-packages/xlsxwriter/worksheet.py:832: UserWarning: Ignoring URL 'http://inchelement.hk




viagra


30%20pills%20x%20100mg$88.5$2.95%20per%20itemyour%20save:%20$1510%20pills%20x%20100mg$34.49$3.4=
5%20per%20item%2060%20pills%20x%20100mg$141$2.35%20per%20itemyour%20save:%20$6610%20pills%20x%2050mg$=
30.22$3.03%20per%20item%2060%20pills%20x%2050mg$111.65$1.86%20per%20itemyour%20save:%20$7030%20pi=
lls%20x%2050mg$60.23$2.01%20per%20itemyour%20save:%20$3090%20pills%20x%20100mg$176.4$1.96%20per=
%20itemyour%20save:%20$134


viagra%20soft%20tabs

%2010%20pills%20x%20100mg%20%20$37.94%20%20$3.8%20per%20item%20%20%20%20%2010%20pills%20x%2050mg%20%20$33.25%20%20$3.33=
%20per%20item%20%20%20%20%2060%20pills%20x%2050mg%20%20$123.2%20%20$2.05%20per%20item%20%20your%20save:%20$77%20%2030%20p=
ills%20x%2050mg%20%20$66.28%20%20$2.21%20per%20item%20%20your%20save:%20$34%20%2030%20pills%20x%20100mg%20%20$97.=
35%20%20$3.25%20per%20item%20%20your%20save:%20$16%20%2060%20pil

hello list,

i wonder which user flags can be specified when adding users with the  
net command.

the documentation shows:

[rpc|ads] user add name [password] [-f user flags] [-c comment]

but the user flags are not mentioned anywhere.

are they implemented?

regards,
henrik


In [0]:
from google.colab import files
files.download('structured.xlsx')